In [3]:
import requests
import pattern3.web as web
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
from progressbar import ProgressBar
import pickle

In [ ]:
webpage = 'https://itunes.apple.com/us/genre/podcasts/id26?mt=2'

In [ ]:
def extract_links(webpage):
    try: 
        url = web.URL(webpage)
        bs = BeautifulSoup(url.download(cached = False)) 
        links_page = []
        for link in bs.findAll('a', href=True):
            links_page.append(link['href'])
#             print (link['href'])
    except:
        pass 
    return links_page

In [ ]:
links = extract_links(webpage)

In [ ]:
def detect_patterns(links, pattern):
    pattern_links = [] 
    for link in links:
            if re.search(pattern, link):
                pattern_links.append(link)
                print (link)
    return pattern_links

<br><br/>

### 2. Extract Genre Links 

In [ ]:
pattern = 'https://itunes.apple.com/us/genre/podcasts-'

In [ ]:
genre_links = detect_patterns(links, pattern)

### 3. Extract Popular Podcasts 

In [ ]:
pattern2 = 'https://itunes.apple.com/us/podcast/'

In [ ]:
podcast_links = []

In [ ]:
for genre_link in genre_links:
    all_links = extract_links(genre_link)
    podcast_links.append(detect_patterns(all_links, pattern2))

In [ ]:
podcast_links2 = list(set(np.concatenate(podcast_links).tolist()))

In [ ]:
with open('podcast_links', 'wb') as fp:
    pickle.dump(podcast_links2, fp)

In [7]:
with open ('podcast_links', 'rb') as fp:
    podcast_links2 = pickle.load(fp)


In [33]:
len(podcast_links2)

13656

<br><BR/>

### 4. Get Names, Descriptions 

In [162]:
columns= ['Name', 'Artwork', 'Genre IDs', 'Episode Count', 'Episode Durations','iTunes URL', 'Feed URL', 'Podcast URL', 'Description']
df_podcast = pd.DataFrame([[0, 0, 0, 0, 0, 0, 0, 0, 0]], columns = columns) 

In [163]:
df_podcast

,Name,Artwork,Genre IDs,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description
0,0,0,0,0,0,0,0,0,0


In [165]:
# Get artwork, feedurl genreIDs from the iTunes API: 
def data_from_API(ID):

    url = 'https://itunes.apple.com/lookup?id=' + ID

    try:
        response = requests.get(url) 
        data = response.json()
        
        if data["resultCount"] != 0: 
            for elem in data["results"]:   
                if "kind" in elem.keys(): 
                    if elem["kind"] == "podcast":
                        if "trackCount" in elem.keys():
                            if elem["trackCount"] >= 20: 
                                if "artworkUrl100" in elem.keys():
                                    artwork = elem["artworkUrl100"]
                                if "feedUrl" in elem.keys(): 
                                    rss_feed = elem["feedUrl"]
                                if "genreIds" in elem.keys():
                                    genre_IDs = elem["genreIds"]
                                    
                                    
                                return [artwork, rss_feed, genre_IDs]
                            
                            else: 
                                return None 

                    else:
                        return None 
                          
        else: 
            return None

    except:
        None 

In [164]:
def get_description(webpage):
    
    global df_podcast
    iTunes_URL = webpage
    podcast_id = re.findall('[0-9]+', re.findall('id[0-9]+', iTunes_URL)[0])[0]
    api_data = data_from_API(podcast_id)
    
    if api_data is not None: 
        artwork = api_data[0]
        feedurl = api_data[1]
        genre_IDs = api_data[2]
        
        
    
        try: 

            url = web.URL(webpage)
            bs = BeautifulSoup(url.download(cached = False)) 

            titles = bs.find('div', id='title')
            if titles is not None:
                title = titles.find('h1').getText()


            try:     
                podcast_left = bs.find('div', id='left-stack')
                if podcast_left is not None:
                    links = podcast_left.findAll('a', href=True)
                    if links is not None: 
                        for link in links:                   
                            if link.getText() == "Podcast Website":
                                podcast_link = link['href']

            except:
                podcast_link = 0



            try:              
                description = bs.find('div', 'product-review')
                if description is not None:
                    descr_parag = description.findAll('p')
                    for parag in descr_parag:
                        podcast_description = parag.getText()

            except:
                podcast_description = 0 




            try:        
                track_list = bs.find('div', 'track-list', 'podcast')
                episodes = track_list.findAll('tr', 'podcast-episode')
                episode_count = len(episodes)
                durations = [] 
                for episode in episodes:
                    if episode['preview-duration'] is not None:
                        duration = int(episode['preview-duration']) / 1000 / 60 
                        durations.append(float("{0:.2f}".format(duration)))
                    else:
                        duration = 0 
                        durations.append(duration)
    #             print (number_episodes, durations)    
            except: 
                durations = 0  


            my_row = [title, artwork, genre_IDs, episode_count, durations, iTunes_URL, feedurl, podcast_link, podcast_description]
            my_row_pd = pd.DataFrame([my_row], columns = columns)  
            df_podcast = df_podcast.append(my_row_pd, ignore_index=True)

            return df_podcast

        except: 
            return None 
        
    else: 
        return None 

In [73]:
podcast_links3 = list(set(podcast_links2))

In [167]:
print (len(podcast_links2), len(podcast_links3))

13656 13656


<br> <br/>

In [166]:
pbar = ProgressBar()

In [169]:
for link in pbar(podcast_links2):
    get_description(link)

/Users/odeniz/Downloads/ENTER/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/odeniz/Downloads/ENTER/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
100% |########################################################################|


In [172]:
df_podcast.drop(df_podcast.index[0], inplace=True)
dfpodcast = df_podcast.reset_index(drop=True)

In [174]:
dfpodcast.to_csv("df_popular_podcasts.csv", index=False)